In [1]:
import json
import requests
import datetime
import pandas as pd
import urllib.request
import codecs

In [2]:
#declare index list and months
list_indexes = ['10' , '20' , '40' , 'Total' , 'ETH' , 'Currency' , 'Platform' , 'Application']
weights_API_list = ['TEN' , 'TWENTY' , 'FORTY' , 'TOTAL' , 'ETHEREUM' , 'CURRENCY' , 'PLATFORM' , 'APPLICATION']
old_month = 'september'        #change these every rebalance
new_month = 'october'          #change these every rebalance

#use staticvalues when actually rebalance
#otherwise test as extra safety on undesired changes
#masterfile = 'Test'
masterfile = 'StaticValues'

In [3]:
#read in inputs and declare output variables
indexes = dict.fromkeys(list_indexes)
old_constants = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/All_'+old_month+'.csv' , index_col=[0])
new_values = old_constants * 1
new_values['Mkt Cap'] = old_constants['USD'] * 0
new_values['Adj Mkt Cap'] = old_constants['USD'] * 0
for k in indexes:
    if (k == 'Currency' or k == 'Platform' or k == 'Application'):
        indexes[k] = pd.DataFrame()
        #change call to new filepath
        indexes[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Adj Float'])
        if 'MIOTA' in indexes[k].index:
            indexes[k] = indexes[k].rename({'MIOTA' : 'IOT'})
    else:
        indexes[k] = pd.DataFrame()
        #change call to new filepath
        indexes[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Float' , 'Adj Float'])
        if 'MIOTA' in indexes[k].index:
            indexes[k] = indexes[k].rename({'MIOTA' : 'IOT'})

In [4]:
#print to help check
old_constants

,Divisor,Adj Divisor,USD,BTC,Adj USD,Adj BTC
10,2.621862e+08,1.865867e+08,912.648709,0.086941,1282.427197,0.122167
20,1.795412e+07,1.971897e+07,509.315731,0.048518,463.731855,0.044176
40,6.079181e+07,5.057988e+07,44.633054,0.004252,53.644336,0.005110
Total,6.823173e+08,2.049727e+09,368.071199,0.035063,122.524270,0.011672
ETH,1.033058e+08,1.796709e+08,63.389381,0.006039,36.447138,0.003472
Currency,3.607833e+09,0.000000e+00,56.901424,0.005421,0.000000,0.000000
Platform,2.398850e+09,0.000000e+00,16.366563,0.001559,0.000000,0.000000
Application,3.190370e+08,0.000000e+00,20.654675,0.001968,0.000000,0.000000


In [5]:
#snapshot all prices
url = "https://api.nomics.com/v1/prices?key=8ab88c64570680aeb728a3109e69dd96&exchange_exclude=gdax"
response = (urllib.request.urlopen(url).read())
response = response.decode('utf-8')
d = json.loads(response)
all_prices = pd.DataFrame(d)
all_prices = all_prices.set_index('currency')
all_prices['Price'] = pd.to_numeric(all_prices['price'])

In [6]:
#function to look up price of ticker
def daily_price_historical(symbol,base):
    a = all_prices.loc[symbol]['Price']
    return a

In [7]:
#get BTC price
bitcoin = daily_price_historical('BTC','USD')

In [8]:
#calculate current index values, only do one for sectors (just one weighting scheme)
#different weighting caps and methodology requires different logic here
#print statements are a manual check that weights sum to 100%
for k in indexes:
    #Only one weighting scheme for the sector index so computation is different
    if (k == 'Currency' or k == 'Platform' or k == 'Application'):
        for t in indexes[k].index:
            try:
                df = daily_price_historical(t,'USD')
                indexes[k].loc[t,('Price')] = df
            except:
                print (t , 'no price!')
        indexes[k]['Mkt Cap'] = indexes[k]['Price']*indexes[k]['Adj Float']
        indexes[k]['Weight'] = indexes[k]['Mkt Cap'] / indexes[k]['Mkt Cap'].sum()
        sum_weight = indexes[k]['Weight'].sum()
        #check that weights sum to 100%
        print (k, "%.3f" % sum_weight)
        new_values.loc[k,('Mkt Cap')] = indexes[k]['Mkt Cap'].sum()
        new_values.loc[k,('USD')] = indexes[k]['Mkt Cap'].sum() / new_values.loc[k,('Divisor')]
        new_values.loc[k,('BTC')] = new_values.loc[k,('USD')] / bitcoin
        if (indexes[k]['Price'] < 0.001).any() == True:
            print (k, 'missing ticker!')
    else:
    #other indexes all have market cap and equal weighted variations
        for t in indexes[k].index:
            try:
                df = daily_price_historical(t,'USD')
                indexes[k].loc[t,('Price')] = df
            except:
                print (t , 'no price!')
        indexes[k]['Mkt Cap'] = indexes[k]['Price']*indexes[k]['Float']
        indexes[k]['Weight'] = indexes[k]['Mkt Cap'] / indexes[k]['Mkt Cap'].sum()
        indexes[k]['Equal'] = 1.00 / (len(indexes[k]))
        indexes[k]['Adj Mkt Cap'] = indexes[k]['Price'] * indexes[k]['Adj Float']
        indexes[k]['Adj Weight'] = indexes[k]['Adj Mkt Cap'] / indexes[k]['Adj Mkt Cap'].sum()
        sum_weight = indexes[k]['Weight'].sum()
        sum_equal_weight = indexes[k]['Adj Weight'].sum()
        #check that weights sum to 100%
        print (k, "%.3f" % sum_weight, "%.3f" % sum_equal_weight)
        new_values.loc[k,('Mkt Cap')] = indexes[k]['Mkt Cap'].sum()
        new_values.loc[k,('Adj Mkt Cap')] = indexes[k]['Adj Mkt Cap'].sum()
        new_values.loc[k,('USD')] = indexes[k]['Mkt Cap'].sum() / new_values.loc[k,('Divisor')]
        new_values.loc[k,('Adj USD')] = indexes[k]['Adj Mkt Cap'].sum() / new_values.loc[k,('Adj Divisor')]
        new_values.loc[k,('BTC')] = new_values.loc[k,('USD')] / bitcoin
        new_values.loc[k,('Adj BTC')] = new_values.loc[k,('Adj USD')] / bitcoin
        if (indexes[k]['Price'] < 0.001).any() == True:
            print (k, 'missing ticker!')

20 1.000 1.000
40 1.000 1.000
10 1.000 1.000
Currency 1.000
ETH 1.000 1.000
Application 1.000
Platform 1.000
Total 1.000 1.000


In [9]:
#check that index values match pre-rebalance production values
new_values

,Divisor,Adj Divisor,USD,BTC,Adj USD,Adj BTC,Mkt Cap,Adj Mkt Cap
10,2.621862e+08,1.865867e+08,755.379488,0.089924,1104.526095,0.131489,1.980501e+11,2.060899e+11
20,1.795412e+07,1.971897e+07,445.083177,0.052985,403.124997,0.047990,7.991077e+09,7.949211e+09
40,6.079181e+07,5.057988e+07,36.111256,0.004299,43.040377,0.005124,2.195268e+09,2.176977e+09
Total,6.823173e+08,2.049727e+09,305.189751,0.036331,101.681220,0.012105,2.082363e+11,2.084188e+11
ETH,1.033058e+08,1.796709e+08,51.095854,0.006083,31.135819,0.003707,5.278499e+09,5.594202e+09
Currency,3.607833e+09,0.000000e+00,45.616315,0.005430,0.000000,0.000000,1.645760e+11,0.000000e+00
Platform,2.398850e+09,0.000000e+00,15.355112,0.001828,0.000000,0.000000,3.683461e+10,0.000000e+00
Application,3.190370e+08,0.000000e+00,17.524581,0.002086,0.000000,0.000000,5.590990e+09,0.000000e+00


In [10]:
#purely for personal interest to see market cap of universe change
mkt_cap = new_values.loc['Total']['Divisor'] * new_values.loc['Total']['USD']
mkt_cap = mkt_cap / 1000000000
print ('market cap in billions: %.02f' %mkt_cap)

market cap in billions: 208.24


In [11]:
#read in input and declare output variables
indexes , indexes_old = dict.fromkeys(list_indexes) , dict.fromkeys(list_indexes)
indexes_clean = dict.fromkeys(list_indexes)
floats_temp = dict.fromkeys(list_indexes)
indexes_new , old_weights = dict.fromkeys(list_indexes) , dict.fromkeys(list_indexes)
new_weights , new_weights_clean = dict.fromkeys(list_indexes) , dict.fromkeys(list_indexes)
old_weights_clean , turnover = dict.fromkeys(list_indexes) , dict.fromkeys(list_indexes)
indexes_float_change = dict.fromkeys(list_indexes)
indexes_float_change_publish = dict.fromkeys(list_indexes)
old_constants = new_values
new_constants = old_constants * 0
new_constants = new_constants.drop(['Mkt Cap' , 'Adj Mkt Cap'],axis=1)

#use these column designators later
fields = [0 , 4]
fields_sector = [0, 12]

for k in indexes:
    indexes_float_change[k] = pd.DataFrame()
    indexes_float_change_publish[k] = pd.DataFrame()
    floats_temp[k] = pd.DataFrame()
    if (k == 'Currency' or k == 'Platform' or k == 'Application'):
        indexes[k] = pd.DataFrame()
        #change call to new filepath
        indexes[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_pre_'+new_month+'.csv' , index_col=[0])
        indexes_old[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Adj Float'])
    else:
        indexes[k] = pd.DataFrame()
        #change call to new filepath
        indexes[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_pre_'+new_month+'.csv' , index_col=[0])
        indexes_old[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Float','Adj Float'])
    indexes_new[k] = indexes[k]

In [12]:
#print index membership changes - should match ranking date data
for k in indexes:
    print (k , 'adds' , indexes_new[k].index.difference(indexes_old[k].index).values)
    print (k , 'deletes' , indexes_old[k].index.difference(indexes_new[k].index).values)
    print (k , 'count members:' , len(indexes_new[k]))
    if indexes_new[k].index.duplicated(keep='first').any() == True:
        print ('Duplicates if True' , indexes_new[k].index.duplicated(keep='first'))
    print ("")
sectors = len(indexes_new['Currency'])+len(indexes_new['Platform'])+len(indexes_new['Application'])
print ('Sector count members:' , sectors)

20 adds ['IOT' 'QTUM' 'ZRX']
20 deletes ['MAID' 'MANA' 'MIOTA']
20 count members: 20

40 adds ['AION' 'CVC' 'FUN' 'KNC' 'MAID' 'RCN']
40 deletes ['GNO' 'POWR' 'QKC' 'QTUM' 'WAX' 'ZRX']
40 count members: 40

10 adds []
10 deletes []
10 count members: 10

Currency adds ['IOT']
Currency deletes ['MIOTA']
Currency count members: 19

ETH adds ['AION' 'CVC' 'FUN' 'KNC']
ETH deletes ['GNO' 'MANA' 'POWR' 'WAX']
ETH count members: 20

Application adds ['CVC' 'FUN' 'KNC' 'RCN']
Application deletes ['GNO' 'MANA' 'WAX']
Application count members: 24

Platform adds ['AION']
Platform deletes ['POWR' 'QKC']
Platform count members: 27

Total adds ['AION' 'CVC' 'FUN' 'IOT' 'KNC' 'RCN']
Total deletes ['GNO' 'MANA' 'MIOTA' 'POWR' 'QKC' 'WAX']
Total count members: 70

Sector count members: 70


In [13]:
#main function to calculate new floats and divisors
for k in indexes:
    if (k == 'Currency' or k == 'Platform' or k == 'Application'):    #sectors are seperate due to capping
        for t in indexes[k].index:
            #print t
            try:
                df = daily_price_historical(t,'USD')
                indexes[k].loc[t,('Price')] = df
            except:
                print (t)
        #these sectors capped at 25%
        if (k == 'Platform' or k == 'Application'):
            weight_cap = 0.2500
        #no cap in currency anymore
        else:
            weight_cap = 2.0000
        indexes[k]['Mkt Cap'] = indexes[k]['Price']*indexes[k]['Float']
        indexes[k]['Weight'] = indexes[k]['Mkt Cap'] / indexes[k]['Mkt Cap'].sum()
        indexes[k]['Weight Cap'] = weight_cap
        indexes[k]['Weight1'] = indexes[k]['Weight'] * 1
        #start looping through to handle the 25% capping rate
        for t in indexes[k].index:
            indexes[k].loc[t,('Weight1')] = min(indexes[k].loc[t,('Weight')] , indexes[k].loc[t,('Weight Cap')])
        sum1 = indexes[k]['Weight1'].sum()
        resid1 = 1.000 - sum1
        count1 = (sum(f > weight_cap for f in indexes[k]['Weight1'])) * weight_cap
        to_up1 = sum1 - count1
        mult1 = 1.000 + (resid1/to_up1)
        for t in indexes[k].index:
            if indexes[k].loc[t,('Weight1')] < (weight_cap - 0.000001):
                indexes[k].loc[t,('Weight_step2')] = indexes[k].loc[t,('Weight1')] * mult1
                indexes[k].loc[t,('Weight2')] = min(indexes[k].loc[t,('Weight_step2')] , indexes[k].loc[t,('Weight Cap')])
            else:
                indexes[k].loc[t,('Weight_step2')] = indexes[k].loc[t,('Weight1')]
                indexes[k].loc[t,('Weight2')] = indexes[k].loc[t,('Weight1')]
        sum2 = indexes[k]['Weight2'].sum()
        resid2 = 1.000 - sum2
        count2 = (sum(f >= weight_cap for f in indexes[k]['Weight2'])) * weight_cap
        to_up2 = sum2 - count2
        mult2 = 1.000 + (resid2/to_up2)
        for t in indexes[k].index:
            if indexes[k].loc[t,('Weight2')] < (weight_cap - 0.0000001):
                indexes[k].loc[t,('Weight_step3')] = indexes[k].loc[t,('Weight2')] * mult2
                indexes[k].loc[t,('Weight3')] = min(indexes[k].loc[t,('Weight_step3')] , indexes[k].loc[t,('Weight Cap')])
            else:
                indexes[k].loc[t,('Weight_step3')] = indexes[k].loc[t,('Weight2')]
                indexes[k].loc[t,('Weight3')] = indexes[k].loc[t,('Weight2')]
        sum3 = indexes[k]['Weight3'].sum()
        resid3 = 1.000 - sum3
        count3 = (sum(f >= weight_cap for f in indexes[k]['Weight3'])) * weight_cap
        to_up3 = sum3 - count3
        mult3 = 1.000 + (resid3/to_up3)
        for t in indexes[k].index:
            if indexes[k].loc[t,('Weight3')] <= weight_cap:
                indexes[k].loc[t,('Weight_step4')] = indexes[k].loc[t,('Weight3')] * mult3
                indexes[k].loc[t,('Weight4')] = min(indexes[k].loc[t,('Weight_step4')] , indexes[k].loc[t,('Weight Cap')])
            else:
                indexes[k].loc[t,('Weight_step4')] = indexes[k].loc[t,('Weight3')]
                indexes[k].loc[t,('Weight4')] = indexes[k].loc[t,('Weight3')]
        sum4 = indexes[k]['Weight4'].sum()
        resid4 = 1.000 - sum4
        count4 = (sum(f >= weight_cap for f in indexes[k]['Weight4'])) * weight_cap
        to_up4 = sum4 - count4
        mult4 = 1.000 + (resid4/to_up4)
        print (k, mult4 , to_up4, count4, "%.3f" %resid4, "%.3f" %sum4)
        #done iterating to compute proper adjusted weights
        indexes[k]['AWF'] = indexes[k]['Weight4'] / indexes[k]['Weight']
        indexes[k]['Adj Mkt Cap'] = indexes[k]['Mkt Cap'] * indexes[k]['AWF']
        indexes[k]['Adj Weight'] = indexes[k]['Adj Mkt Cap'] / indexes[k]['Adj Mkt Cap'].sum()
        indexes[k]['Adj Float'] = indexes[k]['Float'] * indexes[k]['AWF']
        sum_equal_weight = indexes[k]['Adj Weight'].sum()
        new_constants.loc[k,('Divisor')] = old_constants.loc[k,('Divisor')] *\
        (indexes[k]['Adj Mkt Cap'].sum() / old_constants.loc[k,('Mkt Cap')])
        new_constants.loc[k,('USD')] = indexes[k]['Adj Mkt Cap'].sum() / new_constants.loc[k,('Divisor')]
        new_constants.loc[k,('BTC')] = new_constants.loc[k,('USD')] / bitcoin
    else:
        for t in indexes[k].index:
            try:
                df = daily_price_historical(t,'USD')
                indexes[k].loc[t,('Price')] = df
            except:
                print (t)
        indexes[k]['Mkt Cap'] = indexes[k]['Price']*indexes[k]['Float']
        indexes[k]['Weight'] = indexes[k]['Mkt Cap'] / indexes[k]['Mkt Cap'].sum()
        indexes[k]['Equal'] = 1.00 / (len(indexes[k]))
        indexes[k]['AWF'] = indexes[k]['Equal'] / indexes[k]['Weight']
        indexes[k]['Adj Mkt Cap'] = indexes[k]['Mkt Cap'] * indexes[k]['AWF']
        indexes[k]['Adj Weight'] = indexes[k]['Adj Mkt Cap'] / indexes[k]['Adj Mkt Cap'].sum()
        indexes[k]['Adj Float'] = indexes[k]['Float'] * indexes[k]['AWF']
        sum_weight = indexes[k]['Weight'].sum()
        sum_equal_weight = indexes[k]['Adj Weight'].sum()
        print (k, "%.3f" %sum_weight, "%.3f" %sum_equal_weight, "%.3f" %indexes[k]['Weight'].max(), "%.3f" %indexes[k]['Weight'].min())
        new_constants.loc[k,('Divisor')] = old_constants.loc[k,('Divisor')] *\
        (indexes[k]['Mkt Cap'].sum() / old_constants.loc[k,('Mkt Cap')])
        new_constants.loc[k,('Adj Divisor')] = old_constants.loc[k,('Adj Divisor')] *\
        (indexes[k]['Adj Mkt Cap'].sum() / old_constants.loc[k,('Adj Mkt Cap')])
        new_constants.loc[k,('USD')] = indexes[k]['Mkt Cap'].sum() / new_constants.loc[k,('Divisor')]
        new_constants.loc[k,('Adj USD')] = indexes[k]['Adj Mkt Cap'].sum() / new_constants.loc[k,('Adj Divisor')]
        new_constants.loc[k,('BTC')] = new_constants.loc[k,('USD')] / bitcoin
        new_constants.loc[k,('Adj BTC')] = new_constants.loc[k,('Adj USD')] / bitcoin

20 1.000 1.000 0.114 0.012
40 1.000 1.000 0.055 0.009
10 1.000 1.000 0.692 0.006
Currency 1.0 1.0 0.0 0.000 1.000
ETH 1.000 1.000 0.265 0.005
Application 1.0 1.0 0.0 0.000 1.000
Platform 1.0 0.5 0.5 0.000 1.000
Total 1.000 1.000 0.645 0.000


In [14]:
#check for ticker issues
for k in indexes:
    #print (k)
    #print (indexes[k].index , indexes[k]['Price'])
    if (indexes[k]['Price'] < 0.001).any() == True:
            print (k, 'missing ticker!' , indexes[k]['Price'])

In [15]:
#check that these match pre-rebalance production values
new_constants

,Divisor,Adj Divisor,USD,BTC,Adj USD,Adj BTC
10,3.372491e+08,2.306428e+08,755.379488,0.089924,1104.526095,0.131489
20,3.470219e+07,3.831407e+07,445.083177,0.052985,403.124997,0.047990
40,8.271044e+07,6.939479e+07,36.111256,0.004299,43.040377,0.005124
Total,8.951256e+08,2.686663e+09,305.189751,0.036331,101.681220,0.012105
ETH,1.288072e+08,2.113808e+08,51.095854,0.006083,31.135819,0.003707
Currency,4.437743e+09,0.000000e+00,45.616315,0.005430,0.000000,0.000000
Platform,4.142801e+09,0.000000e+00,15.355112,0.001828,0.000000,0.000000
Application,4.072263e+08,0.000000e+00,17.524581,0.002086,0.000000,0.000000


In [16]:
#total market cap of new universe - used in blog post
mkt_cap = new_constants.loc['Total']['Divisor'] * new_constants.loc['Total']['USD']
print ('%.2f' %(mkt_cap*0.000000001))

273.18


In [17]:
#bitcoin dominance metric used in blog post
print ('%.2f' %(indexes['Currency'].loc['BTC']['Weight4']*100))

87.08


In [18]:
#create float change csv files - extra data not necessary for actual rebalance
#turnover is quickest check on everything working correctly
for k in indexes:
    if (k == 'Currency' or k == 'Platform' or k == 'Application'):
        indexes_float_change[k]['Weight'] = indexes[k]['Weight4']
        indexes_float_change[k]['Price'] = indexes[k]['Price']
        indexes_float_change[k]['Previous Float'] = indexes_old[k]['Adj Float']
        indexes_float_change[k]['Float'] = indexes[k]['Adj Float']
        indexes_float_change[k]['Float Change'] = indexes[k]['Adj Float'] - indexes_old[k]['Adj Float']
        indexes_float_change[k]['Float Change Percent'] = ((indexes[k]['Adj Float'] / indexes_old[k]['Adj Float']) - 1)*100
        indexes_float_change[k]['Float Change Units per 1 bps'] = indexes_float_change[k]['Float Change'] * 0.0001
        indexes_float_change[k]['Funding'] = indexes_float_change[k]['Float Change Units per 1 bps'] * indexes_float_change[k]['Price']
        indexes_float_change[k]['Funding Units per 1 bps'] = (-indexes_float_change[k]['Funding'].sum()*\
                                                        indexes_float_change[k]['Weight']) / indexes_float_change[k]['Price']
        indexes_float_change[k]['Total Flow Units per 1 bps'] = indexes_float_change[k]['Float Change Units per 1 bps']\
        + indexes_float_change[k]['Funding Units per 1 bps']
        if indexes_float_change[k]['Float Change Percent'].any() > (9.99):
            print (k , 'error!')
    else:
        #print (indexes_float_change[k][indexes_float_change[k].index.duplicated()])
        indexes_float_change[k]['Weight'] = indexes[k]['Weight']
        indexes_float_change[k]['Price'] = indexes[k]['Price']
        indexes_float_change[k]['Previous Float'] = indexes_old[k]['Float']
        indexes_float_change[k]['Float'] = indexes[k]['Float']
        indexes_float_change[k]['Float Change'] = indexes_new[k]['Float'] - indexes_old[k]['Float']
        indexes_float_change[k]['Float Change Percent'] = ((indexes_new[k]['Float'] / indexes_old[k]['Float']) - 1)*100
        indexes_float_change[k]['Float Change Units per 1 bps'] = indexes_float_change[k]['Float Change'] * 0.0001
        indexes_float_change[k]['Funding'] = indexes_float_change[k]['Float Change Units per 1 bps'] * indexes_float_change[k]['Price']
        indexes_float_change[k]['Funding Units per 1 bps'] = (-indexes_float_change[k]['Funding'].sum()*\
                                                        indexes_float_change[k]['Weight']) / indexes_float_change[k]['Price']
        indexes_float_change[k]['Total Flow Units per 1 bps'] = indexes_float_change[k]['Float Change Units per 1 bps']\
        + indexes_float_change[k]['Funding Units per 1 bps']
        if indexes_float_change[k]['Float Change Percent'].any() > 9.99:
            print (k , 'error!')
#indexes_float_change

In [19]:
#open text file containing divisors and float csvs
#change call to new filepath
with open('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/java/com/frobro/bcindex/web/constants/'+masterfile+'.java','r') as constants:
    lines = constants.readlines()

In [20]:
#declare all divisors
ten_divisor = new_constants.loc['10']['Divisor'].astype(str)
ten_divisor_even = new_constants.loc['10']['Adj Divisor'].astype(str)
twenty_divisor = new_constants.loc['20']['Divisor'].astype(str)
twenty_divisor_even = new_constants.loc['20']['Adj Divisor'].astype(str)
forty_divisor = new_constants.loc['40']['Divisor'].astype(str)
forty_divisor_even = new_constants.loc['40']['Adj Divisor'].astype(str)
total_divisor = new_constants.loc['Total']['Divisor'].astype(str)
total_divisor_even = new_constants.loc['Total']['Adj Divisor'].astype(str)
eth_divisor = new_constants.loc['ETH']['Divisor'].astype(str)
eth_divisor_even = new_constants.loc['ETH']['Adj Divisor'].astype(str)
currency_divisor = new_constants.loc['Currency']['Divisor'].astype(str)
platform_divisor = new_constants.loc['Platform']['Divisor'].astype(str)
application_divisor = new_constants.loc['Application']['Divisor'].astype(str)

In [21]:
#crux of program - writing next divisors and csv links
new_csv_file_lines = {8 , 9 , 12 , 18 , 24 , 30 , 36 , 41 , 46 , 51 , 16 , 17,22,23,28,29,34,35,40,45,50}
indexes_csv = {'10' , '20' , '40' , 'ETH' , 'Total' , 'Currency' , 'Platform' , 'Application'}
for l in new_csv_file_lines:
    #print l
    if l == 12:
        lines[l] = '  public static final String MKT_CAP_FILE = "business_rules/10_'+new_month+'_rebal.csv";\n'
    elif l == 8:
        lines[l] = '  public static final double DIVISOR_TEN = '+ten_divisor+';\n'
    elif l == 9:
        lines[l] = '  public static final double DIVISOR_EVEN_TEN = '+ten_divisor_even+';\n'
    elif l == 16:
        lines[l] = '  public static final double DIVISOR_20 = '+twenty_divisor+';\n'
    elif l == 17:
        lines[l] = '  public static final double DIVISOR_EVEN_20 = '+twenty_divisor_even+';\n'
    elif l == 22:
        lines[l] = '  public static final double DIVISOR_40 = '+forty_divisor+';\n'
    elif l == 23:
        lines[l] = '  public static final double DIVISOR_EVEN_40 = '+forty_divisor_even+';\n'
    elif l == 34:
        lines[l] = '  public static final double DIVISOR_TOTAL = '+total_divisor+';\n'
    elif l == 35:
        lines[l] = '  public static final double DIVISOR_EVEN_TOTAL = '+total_divisor_even+';\n'
    elif l == 28:
        lines[l] = '  public static final double DIVISOR_ETHER = '+eth_divisor+';\n'
    elif l == 29:
        lines[l] = '  public static final double DIVISOR_EVEN_ETHER = '+eth_divisor_even+';\n'
    elif l == 40:
        lines[l] = '  public static final double DIVISOR_CURRENCY = '+currency_divisor+';\n'
    elif l == 45:
        lines[l] = '  public static final double DIVISOR_PLATFORM = '+platform_divisor+';\n'
    elif l == 50:
        lines[l] = '  public static final double DIVISOR_APPLICATION = '+application_divisor+';\n'
    elif l == 18:
        lines[l] = '  public static final String MKT_CAP_FILE_20 = "business_rules/20_'+new_month+'_rebal.csv";\n'
    elif l == 24:
        lines[l] = '  public static final String MKT_CAP_FILE_40 = "business_rules/40_'+new_month+'_rebal.csv";\n'
    elif l == 30:
        lines[l] = '  public static final String MKT_CAP_FILE_ETHER = "business_rules/ETH_'+new_month+'_rebal.csv";\n'
    elif l == 36:
        lines[l] = '  public static final String MKT_CAP_FILE_TOTAL = "business_rules/Total_'+new_month+'_rebal.csv";\n'
    elif l == 41:
        lines[l] = '  public static final String MKT_CAP_FILE_CURRENCY = "business_rules/Currency_'+new_month+'_rebal.csv";\n'
    elif l == 46:
        lines[l] = '  public static final String MKT_CAP_FILE_PLATFORM = "business_rules/Platform_'+new_month+'_rebal.csv";\n'
    elif l == 51:
        lines[l] = '  public static final String MKT_CAP_FILE_APPLICATION = "business_rules/Application_'+new_month+'_rebal.csv";\n'

In [22]:
#actually save off the above
#change call to new filepath
with open('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/java/com/frobro/bcindex/web/constants/'+masterfile+'.java','w') as constants:
    constants.writelines( lines )
constants.close()

In [23]:
#write index to csv
new_constants.to_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/All_'+new_month+'.csv')
indexes_final = dict.fromkeys(list_indexes)
for k in indexes:
    #change call to new filepath
    indexes[k].to_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs/'+k+'_'+new_month+'_weights.csv')
    indexes_float_change[k].to_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs/'+k+'_'+new_month+'_float_change.csv')
    if (k == 'Currency' or k == 'Platform' or k == 'Application'):
        indexes_final[k] = indexes[k].drop(['Float' , 'Price' , 'Mkt Cap' , 'Weight' , 'Weight Cap' , 'Weight1' , 'Weight_step2' , 'Weight2' \
        , 'Weight_step3' , 'Weight3' , 'Weight_step4' , 'Weight4' , 'AWF' , 'Adj Mkt Cap' , 'Adj Weight'],axis=1)
    else:
        indexes_final[k] = indexes[k].drop(['Price' , 'Mkt Cap' , 'Weight' , 'Equal' , 'AWF' , 'Adj Mkt Cap' , 'Adj Weight'],axis=1)
    #change call to new filepath
    indexes_final[k].to_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+new_month+'_rebal.csv' , header = False)

In [24]:
indexes['20']

,Float,Price,Mkt Cap,Weight,Equal,AWF,Adj Mkt Cap,Adj Weight,Adj Float
LINK,1.000000e+09,1.746406,1.746406e+09,0.113070,0.05,0.442204,7.722680e+08,0.05,4.422041e+08
XTZ,1.923801e+09,0.913047,1.756520e+09,0.113725,0.05,0.439658,7.722680e+08,0.05,8.458140e+08
TRX,1.041451e+11,0.014734,1.534523e+09,0.099352,0.05,0.503263,7.722680e+08,0.05,5.241233e+10
XMR,2.246608e+07,57.559568,1.293138e+09,0.083723,0.05,0.597205,7.722680e+08,0.05,1.341685e+07
DASH,1.760081e+07,72.087685,1.268801e+09,0.082148,0.05,0.608660,7.722680e+08,0.05,1.071290e+07
ETC,1.945714e+08,4.769560,9.280201e+08,0.060084,0.05,0.832167,7.722680e+08,0.05,1.619160e+08
ZEC,2.092423e+07,39.511771,8.267534e+08,0.053528,0.05,0.934097,7.722680e+08,0.05,1.954526e+07
NEO,1.000000e+08,7.914286,7.914286e+08,0.051241,0.05,0.975790,7.722680e+08,0.05,9.757898e+07
IOT,2.779530e+09,0.269114,7.480102e+08,0.048429,0.05,1.032430,7.722680e+08,0.05,2.869670e+09
ICX,4.105533e+09,0.169260,6.949036e+08,0.044991,0.05,1.111331,7.722680e+08,0.05,4.562606e+09


In [25]:
#get old weights from Bletchley API
weights_API_list = ['TEN' , 'TWENTY' , 'FORTY' , 'TOTAL' , 'ETHEREUM' , 'CURRENCY' , 'PLATFORM' , 'APPLICATION']
endPoint = "https://www.bletchleyindexes.com/api/weight";
for k in weights_API_list:
    data = {'index':k}
    old_weights[k] = pd.DataFrame(columns=['Weight'])
    response = requests.post(endPoint, json=data).text
    old_weights[k]['Weight'] = pd.read_json(response, typ='series' , encoding='utf-8')

In [26]:
#reconcile index names - mapping 
list_indexes = ['10' , '20' , '40' , 'Total' , 'ETH' , 'Currency' , 'Platform' , 'Application']
weights_API_list = ['TEN' , 'TWENTY' , 'FORTY' , 'TOTAL' , 'ETHEREUM' , 'CURRENCY' , 'PLATFORM' , 'APPLICATION']
weights_master = dict.fromkeys(list_indexes)
old_weights['10'] = old_weights['TEN']
old_weights['20'] = old_weights['TWENTY']
old_weights['40'] = old_weights['FORTY']
old_weights['ETH'] = old_weights['ETHEREUM']
old_weights['Currency'] = old_weights['CURRENCY']
old_weights['Platform'] = old_weights['PLATFORM']
old_weights['Application'] = old_weights['APPLICATION']
old_weights['Total'] = old_weights['TOTAL']

In [27]:
#calculate turnover from old and new weights
#old_weights[k] = pd.read_csv('C:\Users\stoin\Desktop\Mike\Jupyter\Inputb\\'+k+'_'+old_month+'_weights.csv' , index_col=[0] , usecols=fields)
sectors = ['Application' , 'Platform' , 'Currency']
for k in indexes:
    if k in sectors:
        #change call to new filepath
        new_weights[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs/'+k+'_'+new_month+'_weights.csv' , index_col=[0] , usecols=fields_sector)
        new_weights[k]['Weight'] = new_weights[k]['Weight4']
    else:
        fields = [0,4]
        #change call to new filepath
        new_weights[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs/'+k+'_'+new_month+'_weights.csv' , index_col=[0] , usecols=fields)
turnover_summary = pd.DataFrame(index=list_indexes , columns=['Turnover'])
for k in indexes:
    adds = indexes_new[k].index.difference(indexes_old[k].index).values
    deletes = indexes_old[k].index.difference(indexes_new[k].index).values
    turnover[k] = pd.DataFrame(columns=['Turnover'])
    for t in adds:
        new_ticker = {'Weight': [0]}
        new_add = pd.DataFrame(data = new_ticker)
        old_weights[k] = old_weights[k].append([new_add])
        old_weights[k].index = old_weights[k].index[:-1].append(pd.Index([t]))
    for t in deletes:
        new_ticker = {'Weight': [0]}
        new_delete = pd.DataFrame(data = new_ticker)
        new_weights[k] = new_weights[k].append([new_delete])
        new_weights[k].index = new_weights[k].index[:-1].append(pd.Index([t]))
    new_weights[k] = new_weights[k].fillna(0)
    new_weights_clean[k] = pd.DataFrame(columns=['Weight'])
    new_weights_clean[k]['Weight'] = new_weights[k]['Weight']
    old_weights[k] = old_weights[k].fillna(0)
    old_weights_clean[k] = old_weights[k]
    turnover[k]['Turnover'] = (new_weights_clean[k]['Weight'] - old_weights_clean[k]['Weight']).abs()
    weights_master[k] = pd.DataFrame()
    weights_master[k]['New Weight'] = new_weights_clean[k]['Weight']
    weights_master[k]['Old Weight'] = old_weights_clean[k]['Weight']
    weights_master[k]['Turnover'] = turnover[k]['Turnover']
    weights_master[k] = weights_master[k].fillna(0)
    turnover_summary.loc[k,('Turnover')] = turnover[k]['Turnover'].abs().sum()/2

ValueError: cannot reindex from a duplicate axis

In [ ]:
#save off turnover and float files with local duplicates
upload_weights = pd.DataFrame()
for k in indexes:
    turnover_summary.rename(index={k:k+' Index'},inplace=True)
upload_weights = pd.concat([turnover_summary ])
upload_floats = pd.DataFrame()
for k in indexes:
    index_name = pd.DataFrame(columns = ['New Weight' , 'Old Weight' , 'Turnover'])
    index_name.loc[k] = ['New Weight' , 'Old Weight' , 'Turnover']
    index_name.rename(index={k:k+' Index'},inplace=True)
    upload_weights = upload_weights.append(index_name)
    upload_weights = pd.concat([upload_weights , weights_master[k]])
    index_float_name = pd.DataFrame(columns = ['Float','Float Change','Float Change Percent' ,\
                                               'Float Change Units per 1 bps','Funding Units per 1 bps','Total Flow Units per 1 bps'])
    index_float_name.loc[k] = ['Float' , 'Float Change' , 'Float Change Percent' , 'Float Change Units per 1 bps','Funding Units per 1 bps',\
                              'Total Flow Units per 1 bps']
    index_float_name.rename(index={k:k+' Index'},inplace=True)
    upload_floats = upload_floats.append(index_float_name)
    indexes_float_change_publish[k] = indexes_float_change[k].drop(['Previous Float', 'Weight','Price','Funding'],axis=1)
    upload_floats = pd.concat([upload_floats , indexes_float_change_publish[k]])
upload_weights
#change call to new filepath
upload_weights.to_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs/new_weights_'+new_month+'.csv')
upload_weights.to_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-view/src/main/resources/static/weights/'+new_month+'_2019.csv')
upload_floats.to_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs/new_floats_'+new_month+'.csv')
upload_floats.to_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-view/src/main/resources/static/weights/'+new_month+'_floats_2019.csv')